# Endemo 2 - Useful Energy Demand Model Rework

### Create Input Manager

In [1]:
import input
import utility as uty

# create input manager to read all input
input_manager = input.Input()

specific demand historical: steel
Feste fossile Brennstoffe Europaeische Union - 27 Laender (ab 2020) [(1990.0, 447486.031), (1991.0, 388064.68), (1992.0, 337535.84), (1993.0, 273771.878), (1994.0, 287032.22), (1995.0, 311392.357), (1996.0, 290461.127), (1997.0, 295962.098), (1998.0, 266812.159), (1999.0, 211652.934), (2000.0, 223626.048), (2001.0, 227004.564), (2002.0, 207838.44), (2003.0, 245366.757), (2004.0, 216846.439), (2005.0, 216652.705), (2006.0, 233038.086), (2007.0, 202017.111), (2008.0, 163253.028), (2009.0, 128917.0), (2010.0, 123875.508), (2011.0, 167247.473), (2012.0, 140200.116), (2013.0, 137592.119), (2014.0, 121953.978), (2015.0, 130528.283), (2016.0, 132962.035), (2017.0, 133180.489), (2018.0, 121028.781)]
Feste fossile Brennstoffe Europaeische Union - 28 Laender (2013-2020) [(1990.0, 476116.408), (1991.0, 415899.229), (1992.0, 360041.04), (1993.0, 291986.513), (1994.0, 310952.27), (1995.0, 335674.167), (1996.0, 313660.507), (1997.0, 319908.083), (1998.0, 290975.219)

In [2]:
# check if the right things were read

print(uty.str_dict(input_manager.industry_input.active_products))




Dictionary: {steel: 
	Specific Consumption: 
Dictionary: {all: SC(electricity=1.4, heat=18.13, hydrogen=0, max_subst_h2=0), }
	 BAT: 
Dictionary: {all: BAT(electricity=1.3613483146067418, heat=1.2386516853932585), }
	 Historical: 
Dictionary: {Belgium: [(1967.0, 9712.0), (1968.0, 11568.0), (1969.0, 12832.0), (1970.0, 12607.0), (1971.0, 12445.0), (1972.0, 14532.0), (1973.0, 15522.0), (1974.0, 16227.0), (1975.0, 11582.0), (1976.0, 12145.0), (1977.0, 11346.0), (1978.0, 12692.0), (1979.0, 13535.0), (1980.0, 12422.0), (1981.0, 12377.0), (1982.0, 9992.0), (1983.0, 10154.0), (1984.0, 11300.0), (1985.0, 10683.0), (1986.0, 9713.0), (1987.0, 9783.0), (1988.0, 11217.0), (1989.0, 10948.0), (1990.0, 11414.0), (1991.0, 11348.0), (1992.0, 10331.0), (1993.0, 10172.0), (1994.0, 11331.0), (1995.0, 11606.0), (1996.0, 10818.0), (1997.0, 10739.0), (1998.0, 11425.0), (1999.0, 10931.0), (2000.0, 11636.0), (2001.0, 10762.0), (2002.0, 11343.0), (2003.0, 11114.0), (2004.0, 11698.0), (2005.0, 10420.0), (2006.0,

### Fill countries with information

In [3]:
import country as cty

countries = []

for country_name in input_manager.ctrl.general_settings.active_countries:
    countries.append(cty.Country(country_name, input_manager))



### Get Prognosis


In [4]:
for country in countries:
    print(country.get_name() + ": " + str(country.calculate_demand(2025)))

NotImplementedError: 

## Test Utility functions
### Test Regression

In [ ]:
import utility as uty

# Test Linear Regression

x = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011]
y = [1258.809906,1259.17278,1259.17278,1111.120188,1063.946568,1124.9094,1100.233968,1100.233968,1095.153732]
data = list(zip(x, y))

uty.linear_regression(data, visualize=True)

# Test Quadratic Regression

x = [6, 9, 12, 12, 15, 21, 24, 24, 27, 30, 36, 39, 45, 48, 57, 60]
y = [12, 18, 30, 42, 48, 78, 90, 96, 96, 90, 84, 78, 66, 54, 36, 24]
data = list(zip(x, y))

uty.quadratic_regression(data, visualize=True)


### Test filter out NaN and Inf

In [ ]:
import numpy as np
import math
import utility as uty

x = [6, 9, 12, 12, 15, 21, 24, 24, 27, 30, 36, 39, 45, 48, 57, 60]
y = [12, np.NaN, np.inf, 42, 48, 78, 90, 96, 96, 90, 84, 78, 66, 54, 36, 24]
data = list(zip(x, y))

uty.filter_out_NaN_and_Inf(data)

### Combining Data on the x axis

Attention! If the x axis is ascending for both data arrays, use ascending_x=True for better performance!

In [ ]:
import utility as uty

a = [(1, 100), (2, 200), (3, 300), (4, 400), (5, 500)]
b = [(2, 2000), (3, 3000), (5, 5000)]

print(uty.combine_data_on_x(a, b, ascending_x=True))

a = [(1, 100), (2, 200), (3, 300), (4, 400), (5, 500)]
b = [(3, 3000),(2, 2000),  (5, 5000)]

print(uty.combine_data_on_x(a, b))


### Zip on X explained

In [ ]:
import utility as uty

a = [(1, 100), (2, 200), (3, 300), (4, 400), (5, 500)]
b = [(2, 2000), (3, 3000), (5, 5000)]

print(list(uty.zip_on_x(a, b)))

In [ ]:
import utility as uty

amount = [(1998, 10), (2001, 12), (2003, 11), (2006, 13)]
population = [(2000, 10000), (2001, 11000), (2002, 12000), (2003, 13000), (2004, 14000), (2005, 14000), (2006, 15000)]
gdp = [(2000, 20), (2001, 30), (2002, 20), (2003, 30), (2004, 40), (2005, 50), (2006, 60)]

zip_amount_pop = list(uty.zip_on_x(amount, population))

print("zipped amount & population: " + str(zip_amount_pop))
print("Transformed amount to per capita: " + str(list(map(lambda arg: (arg[0][0], arg[0][1] / arg[1][1]), zip_amount_pop))))